In [1]:
import os
import json
import sys
import great_expectations as ge
import great_expectations.jupyter_ux
from datetime import datetime
import math
import pandas as pd
os.chdir('/Users/mparayil/Desktop/Development/dsa-data-workflows/grtexp_agero_dsa/great_expectations')

2020-03-02T01:29:15-0500 - INFO - Great Expectations logging enabled at INFO level by JupyterUX module.


In [2]:
import ge_prod.ge_data_access as gda
import ge_prod.queries as queries

In [3]:
rule_query = queries.queries.get('zip_code_lookup').get('create_expectations')

In [4]:
rule_query

'select * from zip_code_lookup;'

# Author Expectations



[**Watch a short tutorial video**](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#video)

[**Read more in the tutorial**](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations)

**Reach out for help on** [**Great Expectations Slack**](https://tinyurl.com/great-expectations-slack)


### Get a DataContext object
[Read more in the tutorial](https://great-expectations.readthedocs.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#get-datacontext-object)




In [5]:
context = ge.data_context.DataContext()

2020-03-02T01:29:43-0500 - INFO - Using project config: /Users/mparayil/Desktop/Development/dsa-data-workflows/grtexp_agero_dsa/great_expectations/great_expectations.yml


### List data assets in your project

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#data-assets)


In [6]:
great_expectations.jupyter_ux.list_available_data_asset_names(context)

Inspecting your data sources. This may take a moment...


#### Pick one of the data asset names above and use as the value of data_asset_name argument below

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#get-batch)


### Specify data_asset & expectation_suite_name

In [7]:
data_asset_name = 'zip_code_lookup'
normalized_data_asset_name = context.normalize_data_asset_name(data_asset_name)
print(normalized_data_asset_name)

NormalizedDataAssetName(datasource='agero_dsa_pandas', generator='default', generator_asset='zip_code_lookup')


### Create a new empty expectation suite

In [8]:
expectation_suite_name = 'warnings'
context.create_expectation_suite(data_asset_name=normalized_data_asset_name, expectation_suite_name=expectation_suite_name,
                                overwrite_existing=True)

{'data_asset_name': 'agero_dsa_pandas/default/zip_code_lookup',
 'expectation_suite_name': 'warnings',
 'meta': {'great_expectations.__version__': '0.8.8'},
 'expectations': []}

In [9]:
context.list_expectation_suite_keys()

[{'data_asset_name': agero_dsa_pandas/default/customer_experience,
  'expectation_suite_name': 'warnings_2019Q3-Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_outreach,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_outreach,
  'expectation_suite_name': 'warnings_dec2019'},
 {'data_asset_name': agero_dsa_pandas/default/network_outreach,
  'expectation_suite_name': 'warnings'},
 {'data_asset_name': agero_dsa_pandas/default/customer_complaints,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_claims,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_claims,
  'expectation_suite_name': 'warnings_dec2019'},
 {'data_asset_name': agero_dsa_pandas/default/service_progress,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/zip_code_lookup,
  'expectation_suite_name': 'warnings'}]

### Get batch to create expectations against

In [11]:
rule_df = gda.snowflake_connector_to_df(rule_query)
# rule_df.to_pickle('temp_data/network_claims_2019Q4.pkl')

In [12]:
rule_df.to_pickle('temp_data/zip_code_lookup_2020-03-02.pkl')

In [13]:
rule_df.shape

(42900, 9)

In [14]:
b_kwargs = {"dataset": rule_df}
batch = context.get_batch(normalized_data_asset_name, expectation_suite_name=expectation_suite_name,
                         batch_kwargs=b_kwargs)

In [15]:
batch.get_row_count()

42900

In [16]:
print(rule_df.shape)

(42900, 9)


In [17]:
[datasource['name'] for datasource in context.list_datasources() if datasource['class_name'] == 'PandasDatasource']

['agero_dsa_pandas']

In [18]:
# getting rule_df batchId & fingerprint
rule_batch_fingerprint = batch.batch_fingerprint
rule_batch_id = batch.batch_id

In [19]:
print('rule_batch_fingerprint: ', rule_batch_fingerprint, sep='\n')
print('rule_batch_id: ', rule_batch_id, sep='\n')

rule_batch_fingerprint: 
{'partition_id': '20200302T063258.647692Z', 'fingerprint': '425ebd75b748e9cd23d8b666e026c596'}
rule_batch_id: 
{'timestamp': 1583130774.648692, 'PandasInMemoryDF': True, 'fingerprint': 'd4a68a045ef8eda6b67a9fe0e0812f63'}


## Author Expectations

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#create-expectations)

See available expectations in the [expectation glossary](https://docs.greatexpectations.io/en/latest/glossary.html?utm_source=notebook&utm_medium=create_expectations)


### Dataset exploration & understanding of fields to ensure rules reflect behavior of data
- Validating columns to exist in table shape
- Expected column count in table shape
- Expected set values to be seen in given column
- Expected columns to have null or non-null values X percentage of the time
- Expect column values to be of certain data type(s)
- Placing max and min values limits on numerical columns
- Average or median column value to be within a certain range
- Expecting column A to be large/less than column B

### 1. Validating to see if every column exists in table

In [20]:
# add more expectations here
column_names = batch.get_table_columns()
column_names

['ZIP_CODE',
 'CITY',
 'STATE_CODE',
 'METRO_NAME',
 'METRO_TYPE',
 'COUNTY',
 'LATITUDE',
 'LONGITUDE',
 'TIMEZONE']

In [21]:
colnames = list(batch.columns)
# colnames.sort()

In [22]:
print(colnames)

['ZIP_CODE', 'CITY', 'STATE_CODE', 'METRO_NAME', 'METRO_TYPE', 'COUNTY', 'LATITUDE', 'LONGITUDE', 'TIMEZONE']


In [24]:
master_column_names = ['ZIP_CODE', 'CITY', 'STATE_CODE', 'METRO_NAME', 'METRO_TYPE', 'COUNTY', 'LATITUDE', 'LONGITUDE', 'TIMEZONE']

In [25]:
len(master_column_names)

9

In [26]:
len(column_names)

9

In [27]:
# Ensuring columns to exist
for col in master_column_names:
    print(col + ':', batch.expect_column_to_exist(col, result_format='BASIC', catch_exceptions=True), sep='\n')

ZIP_CODE:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
CITY:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
STATE_CODE:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
METRO_NAME:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
METRO_TYPE:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
COUNTY:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
LATITUDE:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
LONGITUDE:
{'success': True, 'exception_info': {'raised_exception': False, 'excepti

### 2. Validating column count in table is always the same

In [28]:
print('# of columns in customer_complaints: ', len(column_names))

# of columns in customer_complaints:  9


In [29]:
print('# of columns in {}: '.format('network_claims'), len(master_column_names), '\n')
if len(column_names) == len(master_column_names):
    print(batch.expect_table_column_count_to_equal(len(column_names), result_format='SUMMARY'))
else:
    print(batch.expect_table_column_count_to_equal(len(master_column_names), result_format='SUMMARY'))

# of columns in network_claims:  9 

{'success': True, 'result': {'observed_value': 9}}


### 3. Checking which columns should not have null values

In [30]:
# identifying which columns should not be null
print(column_names)

['ZIP_CODE', 'CITY', 'STATE_CODE', 'METRO_NAME', 'METRO_TYPE', 'COUNTY', 'LATITUDE', 'LONGITUDE', 'TIMEZONE']


In [31]:
rule_df.isnull().sum()

ZIP_CODE         0
CITY             0
STATE_CODE       0
METRO_NAME       0
METRO_TYPE       0
COUNTY        3060
LATITUDE      1643
LONGITUDE     1643
TIMEZONE      1914
dtype: int64

In [40]:
# Separating null & non-null columns
null_cols = list(batch.isnull().sum()[batch.isnull().sum() > 0].keys())
not_null_cols = list(batch.isnull().sum()[batch.isnull().sum() == 0].keys())

In [41]:
print('Viewing column null value counts: ', batch.isnull().sum(), sep='\n')

Viewing column null value counts: 
ZIP_CODE         0
CITY             0
STATE_CODE       0
METRO_NAME       0
METRO_TYPE       0
COUNTY        3060
LATITUDE      1643
LONGITUDE     1643
TIMEZONE      1914
dtype: int64


In [42]:
not_null_cols.sort()

In [43]:
print(not_null_cols)

['CITY', 'METRO_NAME', 'METRO_TYPE', 'STATE_CODE', 'ZIP_CODE']


In [44]:
null_cols.sort()

In [45]:
print(null_cols)

['COUNTY', 'LATITUDE', 'LONGITUDE', 'TIMEZONE']


In [46]:
# checking for all columns that shouldn't be null are not
for col in not_null_cols:
    print(col, '\n', batch.expect_column_values_to_not_be_null(col, result_format='BASIC'))

CITY 
 {'success': True, 'result': {'element_count': 42900, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
METRO_NAME 
 {'success': True, 'result': {'element_count': 42900, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
METRO_TYPE 
 {'success': True, 'result': {'element_count': 42900, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
STATE_CODE 
 {'success': True, 'result': {'element_count': 42900, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
ZIP_CODE 
 {'success': True, 'result': {'element_count': 42900, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}


In [48]:
print(f"capturing the following columns to not be null: {not_null_cols}", sep='\n')

capturing the following columns to not be null: ['CITY', 'METRO_NAME', 'METRO_TYPE', 'STATE_CODE', 'ZIP_CODE']


### 4. Validating columns to have null values
- **columns to check:**
    - 'ASSIGNED_SOURCE_DETAILS', 'ASSIGNED_TIME_EASTERN', 'ASSIGNED_TIME_LOCAL', 'ASSIGNED_TIME_UTC',
'COMPLETED_LATITUDE', 'COMPLETED_LONGITUDE', 'COMPLETED_SOURCE_DETAILS', 'COMPLETED_TIME_LOCAL',
'DISPATCHER_PROFILE_ID', 'DRIVER_PROFILE_ID', 'ENROUTE_LATITUDE', 'ENROUTE_LONGITUDE',
'ENROUTE_SOURCE_DETAILS', 'ENROUTE_TIME_EASTERN', 'ENROUTE_TIME_LOCAL','ENROUTE_TIME_UTC',	                    'ONSCENE_LATITUDE', 'ONSCENE_LONGITUDE', 'ONSCENE_SOURCE_DETAILS', 'ONSCENE_TIME_EASTERN',
'ONSCENE_TIME_LOCAL', 'ONSCENE_TIME_UTC', 'SERVICE_TIME_EASTERN', 'SERVICE_TIME_LOCAL',
'SERVICE_TIME_UTC', 'STATUS_SOURCE', 'TOW_ARRIVED_LATITUDE', 'TOW_ARRIVED_LONGITUDE',
'TOW_ARRIVED_SOURCE_DETAILS', 'TOW_ARRIVED_TIME_EASTERN', 'TOW_ARRIVED_TIME_LOCAL',
'TOW_ARRIVED_TIME_UTC', 'TOW_IN_PROGRESS_LATITUDE', 'TOW_IN_PROGRESS_LONGITUDE',
'TOW_IN_PROGRESS_SOURCE_DETAILS', 'TOW_IN_PROGRESS_TIME_EASTERN', 'TOW_IN_PROGRESS_TIME_LOCAL',
'TOW_IN_PROGRESS_TIME_UTC'

In [49]:
from typing import Union
from great_expectations.dataset import PandasDataset
def get_df_not_null_weights(df: Union[pd.DataFrame, PandasDataset], groupby_col: str, not_null_col: str) -> float:
    """
    Provides specified column's weight/percentage for it not to be null.

    Parameters
    -----------
    df: pd.DataFrame or great_expectations.dataset.PandasDataset
        dataframe object to look at
    groupby_col: str
        grouping column string to groupby dataframe on when looking at specified column in next parameter
    not_null_col: str
        column used from dataframe to calculate safe weight thresholds of when it would be not null

    Returns
    ------------
    float
        Not null weight of specified column lowered by 5% after looking at the 10% quartile
    """

    df_group = df.groupby(df[groupby_col].dt.date)
    df_group = df_group.apply(lambda x: x[not_null_col].notnull().mean())

    adjusted_weight = df_group.quantile(0.1, interpolation='lower')
    return adjusted_weight.round(4)

In [50]:
from typing import Union
from great_expectations.dataset import PandasDataset
def get_df_not_null_weights(df: Union[pd.DataFrame, PandasDataset], groupby_col: str, not_null_col: str) -> float:
	"""
	Provides specified column's weight/percentage for it not to be null.

	Parameters
	-----------
	df: pd.DataFrame or great_expectations.dataset.PandasDataset
		dataframe object to look at
	groupby_col: str
		grouping column string to groupby dataframe on when looking at specified column in next parameter
	not_null_col: str
		column used from dataframe to calculate safe weight thresholds of when it would be not null

	Returns
	------------
	float
		Not null weight of specified column lowered by 5% after looking at the 10% quartile
	"""
	
	df_group = df.groupby(df[groupby_col].dt.date)
	df_group = df_group.apply(lambda x: x[not_null_col].notnull().mean())

	base_weight = df_group.quantile(0.1, interpolation='midpoint')
	adjusted_weight = (base_weight - 0.009)
	if adjusted_weight < 0.005:
		final_weight = base_weight.round(4)
	else:
		final_weight = adjusted_weight.round(4)
	return float(final_weight)

In [51]:
for col in null_cols:
    w = get_df_not_null_weights(rule_df, 'COMPLETED_TIME_UTC', col)
    print(col, w, sep='\n')

KeyError: 'COMPLETED_TIME_UTC'

In [124]:
# calculating weight for columns of how often they should be null
original_null_percents = dict(1 -(batch.isnull().sum() / len(batch))[batch.isnull().sum() / len(batch) > 0])
    
print('original_null_percents', original_null_percents, sep='\n')

# lowering weights by one thousandth of decimal
adjusted_null_percents = {}
for key, weight in original_null_percents.items():
    adjusted_null_percents[key] = (weight - 0.01).round(3)
    
print('---------------------------------------')
print('adjusted_null_weights:')
print(not_null_weights)
# original_null_percents = {k:round(v, 3) for k, v in original_null_percents.items()}
# adjusted_null_percents = {k:round(v, 3) for k, v in not_null_weights.items()}

original_null_percents
{'COUNTY': 0.9286713286713286, 'LATITUDE': 0.9617016317016317, 'LONGITUDE': 0.9617016317016317, 'TIMEZONE': 0.9553846153846154}
---------------------------------------
adjusted_null_weights:
{'COUNTY': 0.919, 'LATITUDE': 0.952, 'LONGITUDE': 0.952, 'TIMEZONE': 0.945}


In [125]:
default_not_null_weights = {'COUNTY': 0.919, 'LATITUDE': 0.952, 'LONGITUDE': 0.952, 'TIMEZONE': 0.945}

In [126]:
for col, weight in default_not_null_weights.items():
    print(col, batch.expect_column_values_to_not_be_null(col, mostly=weight, include_config=True,
                                                           catch_exceptions=True,
                                                           result_format='SUMMARY'), sep='\n')

COUNTY
{'success': True, 'result': {'element_count': 42900, 'unexpected_count': 3060, 'unexpected_percent': 7.1328671328671325, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_not_be_null', 'kwargs': {'column': 'COUNTY', 'mostly': 0.919, 'result_format': 'SUMMARY'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
LATITUDE
{'success': True, 'result': {'element_count': 42900, 'unexpected_count': 1643, 'unexpected_percent': 3.82983682983683, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_not_be_null', 'kwargs': {'column': 'LATITUDE', 'mostly': 0.952, 'result_format': 'SUMMARY'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
LONGITUDE
{'success': True, 'result': {'element_count': 42900, 'unexpected_count': 1643, 'unexpected_percent': 3.82983682983683, 'partial_unexpected_lis

### 5. Expecting column values to be in a set
- COMPLAINT_CATEGORY
- COMPLAINT_REASON
- COMPLAINT_REASON_DETAILS
- COMPLAINT_ORGIN
- CASE_RESOLUTION
- COMPLAINT_TYPE

In [134]:
from snowflake.connector.converter_null import SnowflakeNoConverterToPython
def snowflake_connector_to_df(query: str) -> pd.DataFrame:
    sf_creds = load_credentials()
    ctx = snowflake.connector.connect(
        user=sf_creds['dsa_username'],
        password=sf_creds['dsa_password'],
        account=sf_creds['dsa_account'],
        database=sf_creds['dsa_master_database'],
        schema=sf_creds['dsa_schema'],
        warehouse=sf_creds['dsa_warehouse'],
        role=sf_creds['dsa_etl_role'],
        converter_class=SnowflakeNoConverterToPython
    )
    cur = ctx.cursor()
    try:
        cur.execute(query)
        df = cur.fetch_pandas_all()
    finally:
        cur.close()
    return df

In [135]:
from snowflake.connector.converter_null import SnowflakeNoConverterToPython
def snowflake_connector_to_df(query: str) -> pd.DataFrame:
    sf_creds = gda.load_credentials()
    ctx = snowflake.connector.connect(
        user=sf_creds['dsa_username'],
        password=sf_creds['dsa_password'],
        account=sf_creds['dsa_account'],
        database=sf_creds['dsa_master_database'],
        schema=sf_creds['dsa_schema'],
        warehouse=sf_creds['dsa_warehouse'],
        role=sf_creds['dsa_etl_role'],
        converter_class=SnowflakeNoConverterToPython
    )
    cur = ctx.cursor()
    try:
        cur.execute(query)
        df = cur.fetch_pandas_all()
    finally:
        cur.close()
    return df

In [148]:
master_column_names


['CASE_ID',
 'TASK_ID',
 'DRIVER_PROFILE_ID',
 'DISPATCHER_PROFILE_ID',
 'ENROUTE_LATITUDE',
 'ENROUTE_LONGITUDE',
 'ONSCENE_LATITUDE',
 'ONSCENE_LONGITUDE',
 'TOW_IN_PROGRESS_LATITUDE',
 'TOW_IN_PROGRESS_LONGITUDE',
 'TOW_ARRIVED_LATITUDE',
 'TOW_ARRIVED_LONGITUDE',
 'COMPLETED_LATITUDE',
 'COMPLETED_LONGITUDE',
 'STATUS_SOURCE',
 'ASSIGNED_SOURCE_DETAILS',
 'ENROUTE_SOURCE_DETAILS',
 'ONSCENE_SOURCE_DETAILS',
 'TOW_IN_PROGRESS_SOURCE_DETAILS',
 'TOW_ARRIVED_SOURCE_DETAILS',
 'COMPLETED_SOURCE_DETAILS',
 'SERVICE_TIME_EASTERN',
 'SERVICE_TIME_UTC',
 'SERVICE_TIME_LOCAL',
 'ASSIGNED_TIME_EASTERN',
 'ASSIGNED_TIME_UTC',
 'ASSIGNED_TIME_LOCAL',
 'ENROUTE_TIME_EASTERN',
 'ENROUTE_TIME_UTC',
 'ENROUTE_TIME_LOCAL',
 'ONSCENE_TIME_EASTERN',
 'ONSCENE_TIME_UTC',
 'ONSCENE_TIME_LOCAL',
 'TOW_IN_PROGRESS_TIME_EASTERN',
 'TOW_IN_PROGRESS_TIME_UTC',
 'TOW_IN_PROGRESS_TIME_LOCAL',
 'TOW_ARRIVED_TIME_EASTERN',
 'TOW_ARRIVED_TIME_UTC',
 'TOW_ARRIVED_TIME_LOCAL',
 'COMPLETED_TIME_EASTERN',
 'COMPLETE

In [129]:
def get_categorical_columns_values(df: Union[pd.DataFrame, PandasDataset], cols: list, table_name: str) -> dict:
	c_weights = {}
	for col in cols:
		unique_weights = df[col].value_counts(normalize=True) * 100
		c_weights[col] = unique_weights.values.mean().round(5)
		
	cat_weight_dict = {c: w for (c, w) in c_weights.items() if w > 0.9 if df[c].dtypes != bool
	                   if c not in ['TASK_ID', 'task_id'] if c not in ['CLIENT_ID', 'client_id']}
	
	execute_strings = ' '.join(f"SELECT DISTINCT {c_name} FROM {table_name};" for c_name in cat_weight_dict.keys())
	ctx = gda.get_snowflake_connector()
	
	cursor_list = ctx.execute_string(execute_strings, remove_comments=True, return_cursors=True)
	category_col_values = {}
	for cur in cursor_list:
		col_names = ','.join([col[0] for col in cur.description])
		cat_values = [x[0] for x in cur.fetchall() if x[0]]
		category_col_values[col_names] = cat_values
	return category_col_values

In [131]:
dtest = get_categorical_columns_values(rule_df, master_column_names, 'zip_code_lookup')

In [160]:
print(dtest)

{'STATE_CODE': ['MA', 'RI', 'NH', 'NY', 'PA', 'VA', 'ME', 'NJ', 'AE', 'NC', 'GA', 'FL', 'IA', 'MN', 'NE', 'AL', 'OH', 'IN', 'MI', 'WI', 'DC', 'WA', 'KY', 'AZ', 'TN', 'MS', 'DE', 'LA', 'AR', 'TX', 'NM', 'NV', 'ND', 'CO', 'OR', 'AS', 'GU', 'PW', 'MP', 'MH', 'PR', 'MO', 'CT', 'WV', 'KS', 'UT', 'CA', 'IL', 'OK', 'ID', 'WY', 'SD', 'MT', 'HI', 'FM', 'AP', 'AK', 'VI', 'SC', 'VT', 'AA', 'MD'], 'METRO_TYPE': ['Tertiary', 'Secondary', 'Primary'], 'TIMEZONE': ['America/New_York', 'Europe/Berlin', 'America/Chicago', 'America/Denver', 'America/Indiana/Indianapolis', 'America/Detroit', 'America/Menominee', 'Europe/Oslo', 'Europe/Brussels', 'America/Godthab', 'Europe/Copenhagen', 'Europe/Rome', 'Atlantic/Azores', 'Europe/Helsinki', 'Atlantic/Reykjavik', 'Europe/Lisbon', 'America/Halifax', 'America/Toronto', 'Asia/Karachi', 'Africa/Monrovia', 'Africa/Kinshasa', 'Asia/Jerusalem', 'Africa/Cairo', 'Europe/Athens', 'Asia/Amman', 'America/Phoenix', 'America/North_Dakota/Beulah', 'Asia/Seoul', 'America/Adak

In [154]:
{k:len(v) for k, v in dtest.items()}

{'STATE_CODE': 62, 'METRO_TYPE': 3, 'TIMEZONE': 96}

In [150]:
{k:len(v) for k, v in category_col_values.items()}

{'ZIP_CODE': 42900,
 'CITY': 18842,
 'STATE_CODE': 62,
 'METRO_NAME': 398,
 'METRO_TYPE': 3,
 'COUNTY': 3214,
 'LATITUDE': 33487,
 'LONGITUDE': 35770,
 'TIMEZONE': 96}

In [127]:
c_weights = {}
for col in master_column_names:
    unique_weights = rule_df[col].value_counts(normalize=True) * 100
    c_weights[col] = unique_weights.values.mean().round(5)

In [146]:
execute_strings = ' '.join(f"SELECT DISTINCT {c_name} FROM zip_code_lookup;" for c_name in master_column_names)
ctx = gda.get_snowflake_connector()

cursor_list = ctx.execute_string(execute_strings, remove_comments=True, return_cursors=True)
category_col_values = {}
for cur in cursor_list:
    col_names = ','.join([col[0] for col in cur.description])
    cat_values = [x[0] for x in cur.fetchall() if x[0]]
    category_col_values[col_names] = cat_values

In [147]:
category_col_values

{'ZIP_CODE': ['01467',
  '01468',
  '01469',
  '01471',
  '01474',
  '01475',
  '01477',
  '01501',
  '01503',
  '01504',
  '01506',
  '01507',
  '01508',
  '01510',
  '01518',
  '01519',
  '01520',
  '01522',
  '01523',
  '01525',
  '01526',
  '01527',
  '01534',
  '01535',
  '01540',
  '01542',
  '01543',
  '01545',
  '01561',
  '01583',
  '01609',
  '02740',
  '02746',
  '02770',
  '02815',
  '02823',
  '02825',
  '02833',
  '01560',
  '01571',
  '01580',
  '01582',
  '01588',
  '02743',
  '02745',
  '02760',
  '02761',
  '02762',
  '02802',
  '02809',
  '02828',
  '02889',
  '02894',
  '02910',
  '03042',
  '03111',
  '01586',
  '01605',
  '02876',
  '03049',
  '03223',
  '03240',
  '03282',
  '14138',
  '01470',
  '01505',
  '01517',
  '01521',
  '01529',
  '01538',
  '01550',
  '01562',
  '01564',
  '01569',
  '01603',
  '02739',
  '02742',
  '02764',
  '02777',
  '02779',
  '02826',
  '02887',
  '02911',
  '02915',
  '03041',
  '03077',
  '01472',
  '01509',
  '01524',
  '01541'

In [148]:
{k:len(v) for k, v in category_col_values.items()}

{'ZIP_CODE': 42900,
 'CITY': 18842,
 'STATE_CODE': 62,
 'METRO_NAME': 398,
 'METRO_TYPE': 3,
 'COUNTY': 3214,
 'LATITUDE': 33487,
 'LONGITUDE': 35770,
 'TIMEZONE': 96}

In [248]:
execute_strings = ' '.join(f"SELECT DISTINCT {col} FROM customer_complaints;" for col in cat_dict)

In [249]:
execute_strings

'SELECT DISTINCT CASE_RESOLUTION FROM customer_complaints; SELECT DISTINCT COMPLAINT_CATEGORY FROM customer_complaints; SELECT DISTINCT COMPLAINT_ORIGIN FROM customer_complaints; SELECT DISTINCT COMPLAINT_REASON FROM customer_complaints; SELECT DISTINCT COMPLAINT_REASON_DETAILS FROM customer_complaints; SELECT DISTINCT COMPLAINT_TYPE FROM customer_complaints;'

In [162]:
for v in category_col_values.values():
    print(len(v))

42900
18842
62
398
3
3214
33487
35770
96


In [269]:
frames = []
for cur in cursor_list:
    df = cur.fetch_pandas_all()
    frames.append(df)
cat_df = pd.concat(frames)

In [272]:
cat_df3 = cat_df.drop_duplicates()

In [271]:
cat_df.COMPLAINT_CATEGORY.nunique()

12

In [428]:
val_set_list = []

for col in master_categorical_columns:
    list_sets = list(batch.get_column_value_counts(col, sort='count').keys())
    val_set_list.append(list_sets)

In [161]:
for c, v in zip(master_categorical_columns, val_set_list):
    print(c, len(v))

NameError: name 'master_categorical_columns' is not defined

In [163]:
for col, val_set in dtest.items():
    print(col, '\n', batch.expect_column_values_to_be_in_set(col, val_set, result_format='BASIC', 
                                        include_config=True, catch_exceptions=True), '\n')

STATE_CODE 
 {'success': True, 'result': {'element_count': 42900, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_be_in_set', 'kwargs': {'column': 'STATE_CODE', 'value_set': ['MA', 'RI', 'NH', 'NY', 'PA', 'VA', 'ME', 'NJ', 'AE', 'NC', 'GA', 'FL', 'IA', 'MN', 'NE', 'AL', 'OH', 'IN', 'MI', 'WI', 'DC', 'WA', 'KY', 'AZ', 'TN', 'MS', 'DE', 'LA', 'AR', 'TX', 'NM', 'NV', 'ND', 'CO', 'OR', 'AS', 'GU', 'PW', 'MP', 'MH', 'PR', 'MO', 'CT', 'WV', 'KS', 'UT', 'CA', 'IL', 'OK', 'ID', 'WY', 'SD', 'MT', 'HI', 'FM', 'AP', 'AK', 'VI', 'SC', 'VT', 'AA', 'MD'], 'result_format': 'BASIC'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}} 

METRO_TYPE 
 {'success': True, 'result': {'element_count': 42900, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'u

In [431]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### 8. Determine if columns are unique per row
- CASE_ID, TASK_ID

In [164]:
for col in master_column_names:
    print(col, batch.expect_column_values_to_be_unique(col, result_format='SUMMARY', catch_exceptions=True), 
          sep='\n')

ZIP_CODE
{'success': True, 'result': {'element_count': 42900, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': [], 'partial_unexpected_index_list': [], 'partial_unexpected_counts': []}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
CITY
{'success': False, 'result': {'element_count': 42900, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 29115, 'unexpected_percent': 67.86713286713287, 'unexpected_percent_nonmissing': 67.86713286713287, 'partial_unexpected_list': ['Templeton', 'Townsend', 'Groton', 'Groton', 'Westminster', 'Auburn', 'Berlin', 'Blackstone', 'Brookfield', 'Clinton', 'Douglas', 'Grafton', 'Holden', 'Holland', 'Jefferson', 'Lancaster', 'Leicester', 'Linwood', 'Millbury', 'Millville'], 'partial_unexpected_index_list': [1, 2, 3, 4, 6, 10, 11, 12, 14, 18, 20, 23, 24, 25, 26, 27, 28, 29, 31, 32], 'p

In [166]:
print(batch.expect_column_values_to_match_regex('ZIP_CODE', 
                                                '^(\d{5}(-\d{4})?|[A-CEGHJ-NPRSTVXY]\d[A-CEGHJ-NPRSTV-Z] ?\d[A-CEGHJ-NPRSTV-Z]\d)$'))

{'success': True, 'result': {'element_count': 42900, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': []}}


In [171]:
batch.expect_multicolumn_values_to_be_unique(column_list=['CASE_ID', 'TASK_ID'], result_format='SUMMARY',
                                            catch_exceptions=True, include_config=True)

{'success': True,
 'result': {'element_count': 2125493,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 0,
  'unexpected_percent': 0.0,
  'unexpected_percent_nonmissing': 0.0,
  'partial_unexpected_list': [],
  'partial_unexpected_index_list': [],
  'partial_unexpected_counts': []},
 'expectation_config': {'expectation_type': 'expect_multicolumn_values_to_be_unique',
  'kwargs': {'column_list': ['CASE_ID', 'TASK_ID'],
   'result_format': 'SUMMARY'}},
 'exception_info': {'raised_exception': False,
  'exception_message': None,
  'exception_traceback': None}}

### 8. Expecting columns to be certain data type

In [167]:
rule_df.dtypes

ZIP_CODE       object
CITY           object
STATE_CODE     object
METRO_NAME     object
METRO_TYPE     object
COUNTY         object
LATITUDE      float64
LONGITUDE     float64
TIMEZONE       object
dtype: object

In [168]:
for x, y in batch.dtypes.iteritems():
    print(x, y)

ZIP_CODE object
CITY object
STATE_CODE object
METRO_NAME object
METRO_TYPE object
COUNTY object
LATITUDE float64
LONGITUDE float64
TIMEZONE object


In [169]:
zip_code_lookup_data_types = dict(batch.dtypes.iteritems())

In [172]:
for key, val in zip_code_lookup_data_types.items():
    zip_code_lookup_data_types[key] = str(val)

In [174]:
print(zip_code_lookup_data_types)

{'ZIP_CODE': 'object', 'CITY': 'object', 'STATE_CODE': 'object', 'METRO_NAME': 'object', 'METRO_TYPE': 'object', 'COUNTY': 'object', 'LATITUDE': 'float64', 'LONGITUDE': 'float64', 'TIMEZONE': 'object'}


In [130]:
for col, typ in network_claims_data_types.items():
    print(batch.expect_column_values_to_be_of_type(col, typ, result_format='SUMMARY', catch_exceptions=True))

{'success': True, 'result': {'observed_value': 'int64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'int8'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'object_'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'int32'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'float64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'float64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'obser

### Review the expectations

Expectations that were true on this data batch were added. To view all the expectations you added so far about this data asset, do:

In [131]:
batch.get_expectation_suite()

2020-02-24T12:38:40-0500 - INFO - 	128 expectation(s) included in expectation_suite. Omitting 3 expectation(s) that failed when last run; set discard_failed_expectations=False to include them. result_format settings filtered.


{'data_asset_name': 'agero_dsa_pandas/default/network_claims',
 'expectation_suite_name': 'warnings_2019Q4',
 'meta': {'great_expectations.__version__': '0.8.8'},
 'expectations': [{'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDCHARGE_AMOUNT'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDCHARGE_COUNT'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDCHARGE_DETAILS'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDPAY_APPROVED_DATE_EASTERN'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDPAY_APPROVED_DATE_UTC'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDPAY_APPROVED_PAYMENT'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDPAY_COUNT'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'ADDPAY_DETAILS'}},
  {'expectation_type': 'expect_column_to

In [41]:
batch.save_expectation_suite()

2020-01-31T18:26:29-0500 - INFO - 	64 expectation(s) included in expectation_suite. result_format settings filtered.


### You created and saved expectations for at least one of the data assets.

### We will show you how to set up validation - the process of checking if new files of this type conform to your expectations before they are processed by your pipeline's code. 

### Go to [integrate_validation_into_pipeline.ipynb](integrate_validation_into_pipeline.ipynb) to proceed.


